In [21]:
from pvlib import iotools
import pandas as pd
from collections import defaultdict

In [2]:
midc_var_map = {
    'BMS': {
        'ghi': 'Global CMP22 (vent/cor) [W/m^2]',
        'dni': 'Direct NIP [W/m^2]',
        'dhi': 'Diffuse CM22-1 (vent/cor) [W/m^2]',
        'wind_speed': 'Avg Wind Speed @ 33ft [m/s]',
        'air_temperature': 'Tower Dry Bulb Temp [deg C]',
        'relative_humidity': 'Tower RH [%]',
    },
    'UOSMRL': {
        'ghi': 'Global CMP22 [W/m^2]',
        'dni': 'Direct NIP [W/m^2]',
        'dhi': 'Diffuse Schenk [W/m^2]',
        'air_temperature': 'Air Temperature [deg C]',
        'relative_humidity': 'Relative Humidity [%]',
        'wind_speed': 'Avg Wind Speed @ 10m [m/s]',
    },
    'HSU': {
        'ghi': 'Global Horiz [W/m^2]',
        'dni': 'Direct Normal (calc) [W/m^2]',
        'dhi': 'Diffuse Horiz (band corr) [W/m^2]',
    },
    'UTPASRL': {
        'ghi': 'Global Horizontal [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horizontal [W/m^2]',
        'air_temperature': 'CHP1 Temp [deg C]',
    },
    'UAT': {
        'ghi': 'Global Horiz (platform) [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horiz [W/m^2]',
        'air_temperature': 'Air Temperature [deg C]',
        'relative_humidity': 'Rel Humidity [%]',
        'wind_speed': 'Avg Wind Speed @ 3m [m/s]',
    },
    'STAC': {
        'ghi': 'Global Horizontal [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horizontal [W/m^2]',
        'wind_speed': 'Avg Wind Speed @ 10m [m/s]',
        'air_temperature': 'Air Temperature [deg C]',
        'relative_humidity': 'Rel Humidity [%]',
    },
    'UNLV': {
        'ghi': 'Global Horiz [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horiz (calc) [W/m^2]',
        'air_temperature': 'Dry Bulb Temp [deg C]',
        'wind_speed': 'Avg Wind Speed @ 30ft [m/s]',
    },
    'ORNL': {
        'ghi': 'Global Horizontal [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horizontal [W/m^2]',
        'air_temperature': 'Air Temperature [deg C]',
        'relative_humidity': 'Rel Humidity [%]',
        'wind_speed': 'Avg Wind Speed @ 42ft [m/s]',
    },
    'NELHA': {
        'ghi': 'Global Horizontal [W/m^2]',
        'air_temperature': 'Air Temperature [deg C]',
        'wind_speed': 'Avg Wind Speed @ 10m [m/s]',
        'relative_humidity': 'Rel Humidity [%]',
    },
    'ULL': {
        'ghi': 'Global Horizontal [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horizontal [W/m^2]',
        'air_temperature': 'Air Temperature [deg C]',
        'relative_humidity': 'Rel Humidity [%]',
        'wind_speed': 'Avg Wind Speed @ 3m [m/s]',
    },
    'VTIF': {
        'ghi': 'Global Horizontal [W/m^2]',
        'dni': 'Direct Normal [W/m^2]',
        'dhi': 'Diffuse Horizontal [W/m^2]',
        'air_temperature': 'Air Temperature [deg C]',
        'wind_speed': 'Avg Wind Speed @ 3m [m/s]',
        'relative_humidity': 'Rel Humidity [%]'
    },
    'NWTC': {
        'ghi': 'Global PSP [W/m^2]',
        'air_temperature': 'Temperature @ 2m [deg C]',
        'wind_speed': 'Avg Wind Speed @ 10m [m/s]',
        'relative_humidity': 'Relative Humidity [%]',
    },
}



In [3]:
surfrad_url = 'ftp://aftp.cmdl.noaa.gov/data/radiation/surfrad/{site}/{year}/{site}{short_year}{jday}.dat'
solrad_url = surfrad_url.replace('surfrad', 'solrad')


In [17]:
params = """
network,name,latitude,longitude,elevation,network_api_id,network_api_abbreviation,timezone,attribution
NREL MIDC,University of Arizona OASIS,32.229690000000005,-110.95533999999999,786.0,UAT,UA OASIS,Etc/GMT+7,"Andreas, A.; Wilcox, S.; (2010). Observed Atmospheric and Solar Information System (OASIS); Tucson, Arizona (Data); NREL Report No. DA-5500-56494. http://dx.doi.org/10.5439/1052226"
NREL MIDC,University of Oregon SRML,44.0467,-123.0743,133.8,UOSMRL,UO SRML,Etc/GMT+8,"Vignola, F.; Andreas, A.; (2013). University of Oregon: GPS-based Precipitable Water Vapor (Data); NREL Report No. DA-5500-64452. http://dx.doi.org/10.7799/1183467"
NREL MIDC,Solar Radiation Research Laboratory BMS,39.7423,-105.1785,1828.8,BMS,SRRL BMS,Etc/GMT+7,"Andreas, A.; Stoffel, T.; (1981). NREL Solar Radiation Research Laboratory (SRRL): Baseline Measurement System (BMS); Golden, Colorado (Data); NREL Report No. DA-5500-56488. http://dx.doi.org/10.5439/1052221"
NREL MIDC,University of Nevada Las Vegas,36.107,-115.1425,615.0,UNLV,UNLV,Etc/GMT+8,"Andreas, A.; Stoffel, T.; (2006). University of Nevada (UNLV): Las Vegas, Nevada (Data); NREL Report No. DA-5500-56509. http://dx.doi.org/10.5439/1052548"
NREL MIDC,Humboldt State University,40.876,-124.08,36.0,HSU,HSU SoRMS,Etc/GMT+8,"Andreas, A.; Wilcox, S.; (2007). Solar Radiation Monitoring Station (SoRMS): Humboldt State University, Arcata, California (Data); NREL Report No. DA-5500-56515. http://dx.doi.org/10.5439/1052559"
NOAA SURFRAD,Table Mountain Boulder CO,40.12498,-105.2368,1689,TBL,tbl,Etc/GMT+7,"Augustine, J.A., J.J. DeLuisi, and C.N. Long, 2000: SURFRAD–A National Surface Radiation Budget Network for Atmospheric Research. Bull. Amer. Meteor. Soc., 81, 2341–2358, https://doi.org/10.1175/1520-0477(2000)081<2341:SANSRB>2.3.CO;2"
NOAA SURFRAD,Desert Rock NV,36.62373,-116.01947,1007,DRA,dra,Etc/GMT+8,"Augustine, J.A., J.J. DeLuisi, and C.N. Long, 2000: SURFRAD–A National Surface Radiation Budget Network for Atmospheric Research. Bull. Amer. Meteor. Soc., 81, 2341–2358, https://doi.org/10.1175/1520-0477(2000)081<2341:SANSRB>2.3.CO;2"
NOAA SURFRAD,Fort Peck MT,48.307829999999996,-105.1017,634,FPK,fpk,Etc/GMT+7,"Augustine, J.A., J.J. DeLuisi, and C.N. Long, 2000: SURFRAD–A National Surface Radiation Budget Network for Atmospheric Research. Bull. Amer. Meteor. Soc., 81, 2341–2358, https://doi.org/10.1175/1520-0477(2000)081<2341:SANSRB>2.3.CO;2"
NOAA SOLRAD,Albuquerque New Mexico,35.03796,-106.62211,1617,ABQ,abq,America/Denver,"B. B. Hicks, J. J. DeLuisi, D. R. Matt, (1996), The NOAA Integrated Surface Irradiance Study (ISIS).A New Surface Radiation Monitoring Program Bulletin of the American Meteorological Society Volume 77, Issue 12, December 1996 pp. 2857-2864."
NOAA SOLRAD,Hanford California,36.31357,-119.63163999999999,73,HNX,hnx,America/Los_Angeles,"B. B. Hicks, J. J. DeLuisi, D. R. Matt, (1996), The NOAA Integrated Surface Irradiance Study (ISIS).A New Surface Radiation Monitoring Program Bulletin of the American Meteorological Society Volume 77, Issue 12, December 1996 pp. 2857-2864."
NOAA SOLRAD,Salt Lake City Utah,40.7722,-111.95495,1288,SLC,slc,America/Denver,"B. B. Hicks, J. J. DeLuisi, D. R. Matt, (1996), The NOAA Integrated Surface Irradiance Study (ISIS).A New Surface Radiation Monitoring Program Bulletin of the American Meteorological Society Volume 77, Issue 12, December 1996 pp. 2857-2864."
NOAA SOLRAD,Seattle Washington,47.68685,-122.25667,20,SEA,sea,America/Los_Angeles,"B. B. Hicks, J. J. DeLuisi, D. R. Matt, (1996), The NOAA Integrated Surface Irradiance Study (ISIS).A New Surface Radiation Monitoring Program Bulletin of the American Meteorological Society Volume 77, Issue 12, December 1996 pp. 2857-2864."
"""

In [20]:
from io import StringIO
pdf = pd.read_csv(StringIO(params))

In [41]:
start = pd.Timestamp('2018-03-01T00:00Z')
end = pd.Timestamp('2019-06-01T00:00Z')

In [42]:
def read_solrad(site, timestamp):
    url = solrad_url.format(site=site.lower(), year=timestamp.strftime('%Y'), short_year=timestamp.strftime("%y"), 
                           jday=timestamp.strftime('%j'))
    df = iotools.read_solrad(url)
    filter_ = (df['ghi_flag'] & df['dni_flag'] & df['dhi_flag']) == 0
    return df[['ghi', 'dni', 'dhi']][filter_]

def read_surfrad(site, timestamp):
    url = surfrad_url.format(site=site.lower(), year=timestamp.strftime('%Y'), short_year=timestamp.strftime("%y"), 
                             jday=timestamp.strftime('%j'))
    df = iotools.read_surfrad(url)[0]
    filter_ = (df['ghi_flag'] & df['dni_flag'] & df['dhi_flag']) == 0
    return df[['ghi', 'dni', 'dhi']][filter_]

def read_midc(site, timestamp):
    end = timestamp.floor('1d') + pd.Timedelta('1d') - pd.Timedelta('1s')
    df = iotools.read_midc_raw_data_from_nrel(
        site, timestamp, end, variable_map={v: k for k, v in midc_var_map[site].items()})
    return df[['ghi', 'dni', 'dhi']]

In [43]:
failed = defaultdict(list)
data = defaultdict(list)
for _, row in pdf.iterrows():
    name = row['name']
    if row.network == 'NREL MIDC':
        f = read_midc
    elif row.network == 'NOAA SOLRAD':
        f = read_solrad
    elif row.network == 'NOAA SURFRAD':
        f = read_surfrad
    for day in pd.date_range(start=start, end=end, freq='1d'):
        print(name, day)
        try:
            data[name].append(f(row.network_api_id, day))
        except Exception:
            print('failed')
            failed[name].append(day)


University of Arizona OASIS 2018-03-01 00:00:00+00:00
University of Arizona OASIS 2018-03-02 00:00:00+00:00
University of Arizona OASIS 2018-03-03 00:00:00+00:00
University of Arizona OASIS 2018-03-04 00:00:00+00:00
University of Arizona OASIS 2018-03-05 00:00:00+00:00
University of Arizona OASIS 2018-03-06 00:00:00+00:00
University of Arizona OASIS 2018-03-07 00:00:00+00:00
University of Arizona OASIS 2018-03-08 00:00:00+00:00
University of Arizona OASIS 2018-03-09 00:00:00+00:00
University of Arizona OASIS 2018-03-10 00:00:00+00:00
University of Arizona OASIS 2018-03-11 00:00:00+00:00
University of Arizona OASIS 2018-03-12 00:00:00+00:00
University of Arizona OASIS 2018-03-13 00:00:00+00:00
University of Arizona OASIS 2018-03-14 00:00:00+00:00
University of Arizona OASIS 2018-03-15 00:00:00+00:00
University of Arizona OASIS 2018-03-16 00:00:00+00:00
University of Arizona OASIS 2018-03-17 00:00:00+00:00
University of Arizona OASIS 2018-03-18 00:00:00+00:00
University of Arizona OASIS 

University of Arizona OASIS 2018-07-31 00:00:00+00:00
University of Arizona OASIS 2018-08-01 00:00:00+00:00
University of Arizona OASIS 2018-08-02 00:00:00+00:00
University of Arizona OASIS 2018-08-03 00:00:00+00:00
University of Arizona OASIS 2018-08-04 00:00:00+00:00
University of Arizona OASIS 2018-08-05 00:00:00+00:00
University of Arizona OASIS 2018-08-06 00:00:00+00:00
University of Arizona OASIS 2018-08-07 00:00:00+00:00
University of Arizona OASIS 2018-08-08 00:00:00+00:00
University of Arizona OASIS 2018-08-09 00:00:00+00:00
University of Arizona OASIS 2018-08-10 00:00:00+00:00
University of Arizona OASIS 2018-08-11 00:00:00+00:00
University of Arizona OASIS 2018-08-12 00:00:00+00:00
University of Arizona OASIS 2018-08-13 00:00:00+00:00
University of Arizona OASIS 2018-08-14 00:00:00+00:00
University of Arizona OASIS 2018-08-15 00:00:00+00:00
University of Arizona OASIS 2018-08-16 00:00:00+00:00
University of Arizona OASIS 2018-08-17 00:00:00+00:00
University of Arizona OASIS 

University of Arizona OASIS 2018-12-27 00:00:00+00:00
University of Arizona OASIS 2018-12-28 00:00:00+00:00
University of Arizona OASIS 2018-12-29 00:00:00+00:00
University of Arizona OASIS 2018-12-30 00:00:00+00:00
University of Arizona OASIS 2018-12-31 00:00:00+00:00
University of Arizona OASIS 2019-01-01 00:00:00+00:00
University of Arizona OASIS 2019-01-02 00:00:00+00:00
University of Arizona OASIS 2019-01-03 00:00:00+00:00
University of Arizona OASIS 2019-01-04 00:00:00+00:00
University of Arizona OASIS 2019-01-05 00:00:00+00:00
University of Arizona OASIS 2019-01-06 00:00:00+00:00
University of Arizona OASIS 2019-01-07 00:00:00+00:00
University of Arizona OASIS 2019-01-08 00:00:00+00:00
University of Arizona OASIS 2019-01-09 00:00:00+00:00
University of Arizona OASIS 2019-01-10 00:00:00+00:00
University of Arizona OASIS 2019-01-11 00:00:00+00:00
University of Arizona OASIS 2019-01-12 00:00:00+00:00
University of Arizona OASIS 2019-01-13 00:00:00+00:00
University of Arizona OASIS 

University of Arizona OASIS 2019-05-28 00:00:00+00:00
University of Arizona OASIS 2019-05-29 00:00:00+00:00
University of Arizona OASIS 2019-05-30 00:00:00+00:00
University of Arizona OASIS 2019-05-31 00:00:00+00:00
University of Arizona OASIS 2019-06-01 00:00:00+00:00
University of Oregon SRML 2018-03-01 00:00:00+00:00
University of Oregon SRML 2018-03-02 00:00:00+00:00
University of Oregon SRML 2018-03-03 00:00:00+00:00
University of Oregon SRML 2018-03-04 00:00:00+00:00
University of Oregon SRML 2018-03-05 00:00:00+00:00
University of Oregon SRML 2018-03-06 00:00:00+00:00
University of Oregon SRML 2018-03-07 00:00:00+00:00
University of Oregon SRML 2018-03-08 00:00:00+00:00
University of Oregon SRML 2018-03-09 00:00:00+00:00
University of Oregon SRML 2018-03-10 00:00:00+00:00
University of Oregon SRML 2018-03-11 00:00:00+00:00
University of Oregon SRML 2018-03-12 00:00:00+00:00
University of Oregon SRML 2018-03-13 00:00:00+00:00
University of Oregon SRML 2018-03-14 00:00:00+00:00
Un

University of Oregon SRML 2018-08-01 00:00:00+00:00
University of Oregon SRML 2018-08-02 00:00:00+00:00
University of Oregon SRML 2018-08-03 00:00:00+00:00
University of Oregon SRML 2018-08-04 00:00:00+00:00
University of Oregon SRML 2018-08-05 00:00:00+00:00
University of Oregon SRML 2018-08-06 00:00:00+00:00
University of Oregon SRML 2018-08-07 00:00:00+00:00
University of Oregon SRML 2018-08-08 00:00:00+00:00
University of Oregon SRML 2018-08-09 00:00:00+00:00
University of Oregon SRML 2018-08-10 00:00:00+00:00
University of Oregon SRML 2018-08-11 00:00:00+00:00
University of Oregon SRML 2018-08-12 00:00:00+00:00
University of Oregon SRML 2018-08-13 00:00:00+00:00
University of Oregon SRML 2018-08-14 00:00:00+00:00
University of Oregon SRML 2018-08-15 00:00:00+00:00
University of Oregon SRML 2018-08-16 00:00:00+00:00
University of Oregon SRML 2018-08-17 00:00:00+00:00
University of Oregon SRML 2018-08-18 00:00:00+00:00
University of Oregon SRML 2018-08-19 00:00:00+00:00
University o

...

Salt Lake City Utah 2018-07-08 00:00:00+00:00
Salt Lake City Utah 2018-07-09 00:00:00+00:00
Salt Lake City Utah 2018-07-10 00:00:00+00:00
Salt Lake City Utah 2018-07-11 00:00:00+00:00
Salt Lake City Utah 2018-07-12 00:00:00+00:00
Salt Lake City Utah 2018-07-13 00:00:00+00:00
Salt Lake City Utah 2018-07-14 00:00:00+00:00
Salt Lake City Utah 2018-07-15 00:00:00+00:00
Salt Lake City Utah 2018-07-16 00:00:00+00:00
Salt Lake City Utah 2018-07-17 00:00:00+00:00
Salt Lake City Utah 2018-07-18 00:00:00+00:00
Salt Lake City Utah 2018-07-19 00:00:00+00:00
Salt Lake City Utah 2018-07-20 00:00:00+00:00
Salt Lake City Utah 2018-07-21 00:00:00+00:00
Salt Lake City Utah 2018-07-22 00:00:00+00:00
Salt Lake City Utah 2018-07-23 00:00:00+00:00
Salt Lake City Utah 2018-07-24 00:00:00+00:00
Salt Lake City Utah 2018-07-25 00:00:00+00:00
Salt Lake City Utah 2018-07-26 00:00:00+00:00
Salt Lake City Utah 2018-07-27 00:00:00+00:00
Salt Lake City Utah 2018-07-28 00:00:00+00:00
Salt Lake City Utah 2018-07-29 00:

Salt Lake City Utah 2019-01-03 00:00:00+00:00
Salt Lake City Utah 2019-01-04 00:00:00+00:00
Salt Lake City Utah 2019-01-05 00:00:00+00:00
Salt Lake City Utah 2019-01-06 00:00:00+00:00
Salt Lake City Utah 2019-01-07 00:00:00+00:00
Salt Lake City Utah 2019-01-08 00:00:00+00:00
Salt Lake City Utah 2019-01-09 00:00:00+00:00
Salt Lake City Utah 2019-01-10 00:00:00+00:00
Salt Lake City Utah 2019-01-11 00:00:00+00:00
Salt Lake City Utah 2019-01-12 00:00:00+00:00
Salt Lake City Utah 2019-01-13 00:00:00+00:00
Salt Lake City Utah 2019-01-14 00:00:00+00:00
Salt Lake City Utah 2019-01-15 00:00:00+00:00
Salt Lake City Utah 2019-01-16 00:00:00+00:00
Salt Lake City Utah 2019-01-17 00:00:00+00:00
Salt Lake City Utah 2019-01-18 00:00:00+00:00
Salt Lake City Utah 2019-01-19 00:00:00+00:00
Salt Lake City Utah 2019-01-20 00:00:00+00:00
Salt Lake City Utah 2019-01-21 00:00:00+00:00
Salt Lake City Utah 2019-01-22 00:00:00+00:00
Salt Lake City Utah 2019-01-23 00:00:00+00:00
Salt Lake City Utah 2019-01-24 00:

Seattle Washington 2018-03-30 00:00:00+00:00
Seattle Washington 2018-03-31 00:00:00+00:00
Seattle Washington 2018-04-01 00:00:00+00:00
Seattle Washington 2018-04-02 00:00:00+00:00
Seattle Washington 2018-04-03 00:00:00+00:00
Seattle Washington 2018-04-04 00:00:00+00:00
Seattle Washington 2018-04-05 00:00:00+00:00
Seattle Washington 2018-04-06 00:00:00+00:00
Seattle Washington 2018-04-07 00:00:00+00:00
Seattle Washington 2018-04-08 00:00:00+00:00
Seattle Washington 2018-04-09 00:00:00+00:00
Seattle Washington 2018-04-10 00:00:00+00:00
Seattle Washington 2018-04-11 00:00:00+00:00
Seattle Washington 2018-04-12 00:00:00+00:00
Seattle Washington 2018-04-13 00:00:00+00:00
Seattle Washington 2018-04-14 00:00:00+00:00
Seattle Washington 2018-04-15 00:00:00+00:00
Seattle Washington 2018-04-16 00:00:00+00:00
Seattle Washington 2018-04-17 00:00:00+00:00
Seattle Washington 2018-04-18 00:00:00+00:00
Seattle Washington 2018-04-19 00:00:00+00:00
Seattle Washington 2018-04-20 00:00:00+00:00
Seattle Wa

Seattle Washington 2018-09-29 00:00:00+00:00
Seattle Washington 2018-09-30 00:00:00+00:00
Seattle Washington 2018-10-01 00:00:00+00:00
Seattle Washington 2018-10-02 00:00:00+00:00
Seattle Washington 2018-10-03 00:00:00+00:00
Seattle Washington 2018-10-04 00:00:00+00:00
Seattle Washington 2018-10-05 00:00:00+00:00
Seattle Washington 2018-10-06 00:00:00+00:00
Seattle Washington 2018-10-07 00:00:00+00:00
Seattle Washington 2018-10-08 00:00:00+00:00
Seattle Washington 2018-10-09 00:00:00+00:00
Seattle Washington 2018-10-10 00:00:00+00:00
Seattle Washington 2018-10-11 00:00:00+00:00
Seattle Washington 2018-10-12 00:00:00+00:00
Seattle Washington 2018-10-13 00:00:00+00:00
Seattle Washington 2018-10-14 00:00:00+00:00
Seattle Washington 2018-10-15 00:00:00+00:00
Seattle Washington 2018-10-16 00:00:00+00:00
Seattle Washington 2018-10-17 00:00:00+00:00
Seattle Washington 2018-10-18 00:00:00+00:00
Seattle Washington 2018-10-19 00:00:00+00:00
Seattle Washington 2018-10-20 00:00:00+00:00
Seattle Wa

Seattle Washington 2019-03-31 00:00:00+00:00
Seattle Washington 2019-04-01 00:00:00+00:00
Seattle Washington 2019-04-02 00:00:00+00:00
Seattle Washington 2019-04-03 00:00:00+00:00
Seattle Washington 2019-04-04 00:00:00+00:00
Seattle Washington 2019-04-05 00:00:00+00:00
Seattle Washington 2019-04-06 00:00:00+00:00
Seattle Washington 2019-04-07 00:00:00+00:00
Seattle Washington 2019-04-08 00:00:00+00:00
Seattle Washington 2019-04-09 00:00:00+00:00
Seattle Washington 2019-04-10 00:00:00+00:00
Seattle Washington 2019-04-11 00:00:00+00:00
Seattle Washington 2019-04-12 00:00:00+00:00
Seattle Washington 2019-04-13 00:00:00+00:00
Seattle Washington 2019-04-14 00:00:00+00:00
Seattle Washington 2019-04-15 00:00:00+00:00
Seattle Washington 2019-04-16 00:00:00+00:00
Seattle Washington 2019-04-17 00:00:00+00:00
Seattle Washington 2019-04-18 00:00:00+00:00
Seattle Washington 2019-04-19 00:00:00+00:00
Seattle Washington 2019-04-20 00:00:00+00:00
Seattle Washington 2019-04-21 00:00:00+00:00
Seattle Wa

In [44]:
import pickle

In [46]:
with open('site_data.pkl', 'wb') as f: pickle.dump(data, f)

In [49]:
data.keys()

dict_keys(['University of Arizona OASIS', 'University of Oregon SRML', 'Solar Radiation Research Laboratory BMS', 'University of Nevada Las Vegas', 'Humboldt State University', 'Table Mountain Boulder CO', 'Desert Rock NV', 'Fort Peck MT', 'Albuquerque New Mexico', 'Hanford California', 'Salt Lake City Utah', 'Seattle Washington'])

In [53]:
new = defaultdict(list)
for name, dd in failed.items():
    row = pdf[pdf['name'] == name].iloc[0]
    if row.network == 'NREL MIDC':
        f = read_midc
    elif row.network == 'NOAA SOLRAD':
        f = read_solrad
    elif row.network == 'NOAA SURFRAD':
        f = read_surfrad
    for day in dd:
        print(name, day)
        try:
            new[name].append(f(row.network_api_id, day))
        except Exception:
            print('failed')

University of Arizona OASIS 2018-09-03 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-04 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-05 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-06 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-07 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-08 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-09 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-10 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-11 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-12 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-13 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-14 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-15 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-16 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-17 00:00:00+00:00
failed
University of Arizona OASIS 2018-09-18 00:00:00+00:00
failed
University of Arizona OA

In [54]:
combo = {}
for name, dd in data.items():
    combo[name] = pd.concat(dd)

In [59]:
import xarray as xr

In [61]:
list(combo.values())[0]

,ghi,dni,dhi
2018-03-01 00:00:00-07:00,-2.37396,-0.430942,-0.095624
2018-03-01 00:01:00-07:00,-2.35124,-0.430941,-0.256988
2018-03-01 00:02:00-07:00,-2.04454,-0.430938,-0.352610
2018-03-01 00:03:00-07:00,-2.08430,-0.430939,-0.358587
2018-03-01 00:04:00-07:00,-2.07294,-0.430939,-0.358587
...,...,...,...
2019-06-01 23:55:00-07:00,-3.08537,-0.433546,0.000000
2019-06-01 23:56:00-07:00,-3.08538,-0.433546,0.000000
2019-06-01 23:57:00-07:00,-3.08538,-0.433547,0.000000
2019-06-01 23:58:00-07:00,-3.08539,-0.433548,0.000000


In [66]:
multi = pd.MultiIndex.from_product([pdf['name'].values, ['ghi', 'dni', 'dhi']], names=['site', 'variable'])

MultiIndex([(            'University of Arizona OASIS', 'ghi'),
            (            'University of Arizona OASIS', 'dni'),
            (            'University of Arizona OASIS', 'dhi'),
            (              'University of Oregon SRML', 'ghi'),
            (              'University of Oregon SRML', 'dni'),
            (              'University of Oregon SRML', 'dhi'),
            ('Solar Radiation Research Laboratory BMS', 'ghi'),
            ('Solar Radiation Research Laboratory BMS', 'dni'),
            ('Solar Radiation Research Laboratory BMS', 'dhi'),
            (         'University of Nevada Las Vegas', 'ghi'),
            (         'University of Nevada Las Vegas', 'dni'),
            (         'University of Nevada Las Vegas', 'dhi'),
            (              'Humboldt State University', 'ghi'),
            (              'Humboldt State University', 'dni'),
            (              'Humboldt State University', 'dhi'),
            (              'Table Mounta

In [80]:
fulldf = None
for name, df in combo.items():
    multi = pd.MultiIndex.from_product([[name], df.columns], names=['site', 'variable'])
    ndf = df.copy()
    ndf.columns = multi
    if fulldf is None:
        fulldf = ndf
    else:
        fulldf = fulldf.join(ndf)
#ndf = pd.concat(new)

In [81]:
fulldf

site                      University of Arizona OASIS                      \
variable                                          ghi       dni       dhi   
2018-03-01 07:00:00+00:00                    -2.37396 -0.430942 -0.095624   
2018-03-01 07:01:00+00:00                    -2.35124 -0.430941 -0.256988   
2018-03-01 07:02:00+00:00                    -2.04454 -0.430938 -0.352610   
2018-03-01 07:03:00+00:00                    -2.08430 -0.430939 -0.358587   
2018-03-01 07:04:00+00:00                    -2.07294 -0.430939 -0.358587   
...                                               ...       ...       ...   
2019-06-02 06:55:00+00:00                    -3.08537 -0.433546  0.000000   
2019-06-02 06:56:00+00:00                    -3.08538 -0.433546  0.000000   
2019-06-02 06:57:00+00:00                    -3.08538 -0.433547  0.000000   
2019-06-02 06:58:00+00:00                    -3.08539 -0.433548  0.000000   
2019-06-02 06:59:00+00:00                    -3.08537 -0.433546  0.000000   

site                      University of Oregon SRML                \
variable                                        ghi    dni    dhi   
2018-03-01 07:00:00+00:00                       NaN    NaN    NaN   
2018-03-01 07:01:00+00:00                       NaN    NaN    NaN   
2018-03-01 07:02:00+00:00                       NaN    NaN    NaN   
2018-03-01 07:03:00+00:00                       NaN    NaN    NaN   
2018-03-01 07:04:00+00:00                       NaN    NaN    NaN   
...                                             ...    ...    ...   
2019-06-02 06:55:00+00:00                    -0.586  0.175 -0.908   
2019-06-02 06:56:00+00:00                    -0.573  0.208 -0.977   
2019-06-02 06:57:00+00:00                    -0.567  0.231 -1.096   
2019-06-02 06:58:00+00:00                    -0.532  0.286 -1.186   
2019-06-02 06:59:00+00:00                    -0.494  0.298 -1.218   

site                      Solar Radiation Research Laboratory BMS            \
variable                                                      ghi       dni   
2018-03-01 07:00:00+00:00                               -0.417106  0.239967   
2018-03-01 07:01:00+00:00                               -0.438184  0.231977   
2018-03-01 07:02:00+00:00                               -0.451286  0.254658   
2018-03-01 07:03:00+00:00                               -0.450628  0.337912   
2018-03-01 07:04:00+00:00                               -0.444867  0.372193   
...                                                           ...       ...   
2019-06-02 06:55:00+00:00                               -0.735677  0.474665   
2019-06-02 06:56:00+00:00                               -0.727341  0.435518   
2019-06-02 06:57:00+00:00                               -0.669173  0.278928   
2019-06-02 06:58:00+00:00                               -0.589671 -0.137017   
2019-06-02 06:59:00+00:00                               -0.473542 -0.459987   

site                                University of Nevada Las Vegas  ...  \
variable                        dhi                            ghi  ...   
2018-03-01 07:00:00+00:00 -0.383588                            NaN  ...   
2018-03-01 07:01:00+00:00 -0.388527                            NaN  ...   
2018-03-01 07:02:00+00:00 -0.403319                            NaN  ...   
2018-03-01 07:03:00+00:00 -0.433228                            NaN  ...   
2018-03-01 07:04:00+00:00 -0.445273                            NaN  ...   
...                             ...                            ...  ...   
2019-06-02 06:55:00+00:00 -0.730237                       -3.65226  ...   
2019-06-02 06:56:00+00:00 -0.706672                       -3.62885  ...   
2019-06-02 06:57:00+00:00 -0.726322                       -3.65226  ...   
2019-06-02 06:58:00+00:00 -0.625366                       -3.50710  ...   
2019-06-02 06:59:00+00:00 -0.544399                       -3.09037  ...   

site                      Albuquerque New Mexico Hanford California            \
variable       

<xarray.Dataset>
Dimensions:                                             (dim_0: 618578)
Coordinates:
  * dim_0                                               (dim_0) object 1519887600000000000 ... 1559458740000000000
Data variables:
    ('University of Arizona OASIS', 'ghi')              (dim_0) float64 -2.374 ... -3.085
    ('University of Arizona OASIS', 'dni')              (dim_0) float64 -0.4309 ... -0.4335
    ('University of Arizona OASIS', 'dhi')              (dim_0) float64 -0.09562 ... 0.0
    ('University of Oregon SRML', 'ghi')                (dim_0) float64 nan ... -0.494
    ('University of Oregon SRML', 'dni')                (dim_0) float64 nan ... 0.298
    ('University of Oregon SRML', 'dhi')                (dim_0) float64 nan ... -1.218
    ('Solar Radiation Research Laboratory BMS', 'ghi')  (dim_0) float64 -0.4171 ... -0.4735
    ('Solar Radiation Research Laboratory BMS', 'dni')  (dim_0) float64 0.24 ... -0.46
    ('Solar Radiation Research Laboratory BMS', 'dhi')  (

In [107]:
uaval = fulldf.xs('University of Arizona OASIS', axis=1).values

In [181]:
das = {}
for var in fulldf.columns.levels[1]:
    limit = fulldf.reorder_levels([1, 0], axis=1).xs(var, axis=1).astype('float32')
    das[var] = xr.DataArray(limit[fulldf.columns.levels[0]].values, dims=['time', 'site'], coords=[limit.index.values, fulldf.columns.levels[0]])
    das[var].encoding = {'dtype': 'float32', 'zlib': True, 'scale_factor': 0.001}
ds = xr.Dataset(das)

In [182]:
import numpy as np
np.allclose(ds.ghi.sel(site='University of Arizona OASIS').values, fulldf.xs('University of Arizona OASIS', axis=1)['ghi'].values)


True

In [194]:
for col, ser in pdf.set_index('name').iteritems():
    ds[col] = xr.DataArray(ser[ds.site.values], dims=['site'])

In [196]:
ds.to_netcdf('site_data.nc')